In [1]:
# Imports

import tensorflow as tf
import pathlib
import os
import pathlib
import IPython
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time


2024-01-29 03:54:20.148708: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 03:54:20.175482: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 03:54:20.175529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 03:54:20.175557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 03:54:20.180598: I tensorflow/core/platform/cpu_feature_g

In [2]:
# downloading the weights oft the base model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = True, 
    weights = "imagenet",
    classes=1000,
    classifier_activation="softmax",                  
)

2024-01-29 03:54:22.116905: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 03:54:22.152282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 03:54:22.152414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
#saving the downloaded base_model 

saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/SavedBaseModel.h5')

base_model.save(saving_path)

/home/samer/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
#Model

model_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([                                        
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(299, 299, 3))
# x = augmentation(inputs)
# x = rescaling(x)
x = tf.keras.applications.inception_v3.preprocess_input(inputs)
x = base_model(x, training=False)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# outputs = tf.keras.layers.Dense(1000, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, x)

# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

# model.compile(
#     optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
#     loss=tf.losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy'],
#     run_eagerly = True)                                               
  
#  return model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Transfer",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()


# model.fit(                                                               # fitting the whole model for non-trainable base
#     dataset_path,
#     epochs=1,
#     validation_data = dataset_path_val)   
    # validation_data = dataset_path_val)
    #verbose = 1)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 299, 299, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 299, 299, 3)       0         
 da)                                                             
                                                                 
 inception_v3 (Functional)   (None, 1000)              23851784  
                                                                 
Total params: 23851784 (90.99 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 23851784 (90.99 MB)
_________________________________________________________________


In [6]:
# Predicting

train_data_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/trai/')

val_data_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/val/')

data_path_test = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/test/')



model.compile(                                                   # compiling the loaded model
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy'])

dictionary = dict()
counter = 0

for file in os.listdir(os.path.join(val_data_path)):
    image_path = os.path.join(val_data_path, file)

    image = tf.keras.utils.load_img(image_path,
                                    target_size=(299,299))
    input_arr = tf.keras.utils.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    # plt.imshow(image)
    # plt.show()
    predictions = model.predict(input_arr)
    pred = np.argmax(predictions, axis=1)
    dictionary[file] = int(pred[0])
    counter +=1
    if counter % 600 == 0:
        time.sleep(5)

print(dictionary)

 
# Writing to sample.json
with open("/home/samer/Desktop/Beedoo/VizWiz/sample_val.json", "a") as outfile:
    json_object = json.dump(dictionary, outfile)



2024-01-29 03:54:25.895065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902


1/1 [==============================] - 0s 28ms/step
{'VizWiz_val_00004558.jpg': 419, 'VizWiz_val_00002987.jpg': 613, 'VizWiz_val_00004902.jpg': 473, 'VizWiz_val_00002369.jpg': 720, 'VizWiz_val_00004956.jpg': 692, 'VizWiz_val_00003544.jpg': 440, 'VizWiz_val_00004289.jpg': 782, 'VizWiz_val_00003615.jpg': 655, 'VizWiz_val_00003080.jpg': 692, 'VizWiz_val_00006513.jpg': 686, 'VizWiz_val_00006494.jpg': 859, 'VizWiz_val_00005689.jpg': 692, 'VizWiz_val_00002040.jpg': 692, 'VizWiz_val_00002215.jpg': 720, 'VizWiz_val_00001291.jpg': 898, 'VizWiz_val_00007724.jpg': 916, 'VizWiz_val_00001867.jpg': 473, 'VizWiz_val_00003659.jpg': 854, 'VizWiz_val_00002980.jpg': 463, 'VizWiz_val_00004335.jpg': 745, 'VizWiz_val_00005273.jpg': 692, 'VizWiz_val_00002397.jpg': 487, 'VizWiz_val_00006146.jpg': 574, 'VizWiz_val_00003036.jpg': 868, 'VizWiz_val_00006825.jpg': 662, 'VizWiz_val_00004124.jpg': 737, 'VizWiz_val_00000905.jpg': 592, 'VizWiz_val_00005483.jpg': 911, 'VizWiz_val_00007519.jpg': 550, 'VizWiz_val_0000625